In [8]:
#preload some datasets

import sklearn.datasets as datasets
import numpy as np

# Want a variety of datasets such that:
## Num of features are varied but datapoints are similar (to measure speed vs feature size)
## Num of datapoints are varied but features are similar (to measure speed vs dataset size)

## DATASET 1: IRIS FROM SKLEARN
# imported as dictionary {'data':, 'target':, 'target_names':}
iris_dataset = datasets.load_iris()
iris_data = np.array(iris_dataset['data'])
iris_target = np.array(iris_dataset['target'])

## DATASET 2: WINE FROM SKLEARN
wine_dataset = datasets.load_wine()
wine_data = np.array(wine_dataset['data'])
wine_target = np.array(wine_dataset['target'])

## DATASET 3: BREAST CANCER FROM SKLEARN
bcancer_dataset = datasets.load_breast_cancer()
bcancer_data = np.array(bcancer_dataset['data'])
bcancer_target = np.array(bcancer_dataset['target'])


print("iris shape", iris_data.shape)
print("wine shape", wine_data.shape)
print("bcancer shape", bcancer_data.shape)


iris shape (150, 4)
wine shape (178, 13)
bcancer shape (569, 30)


In [9]:
import time

print(time.time())

1588021977.983498


In [28]:
# Implementing K-Means
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
import time
import sklearn.metrics as metrics

def K_Means(features, target, search_grid):
    kmeans = KMeans()
    kmeans_random = RandomizedSearchCV(estimator = kmeans, param_distributions = search_grid)
    time_start = time.time()
    
    search = kmeans_random.fit(features, target)
    k_final = search.best_estimator_
    print('time taken: ', time.time() - time_start)
    print(k_final)
    print(search.best_score_)
    return 

In [29]:
n_clusters = [2,3,4,5,6,7,8,9,10] 
n_init = [2,5,10,20]
max_iter = [300,600,1000]
tol = [1e-4]
precompute_distances = ['auto', True, False]
njobs = [-1]
algorithm = ['auto', 'full', 'elkan']

random_grid = {
    'n_clusters': n_clusters,
    'n_init' : n_init,
    'max_iter' : max_iter,
    'tol' : tol,
    'precompute_distances' : precompute_distances,
    'n_jobs' : njobs,
    'algorithm' : algorithm
               }

In [30]:
K_Means(iris_data, iris_target, random_grid)

C:\Users\miles\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


time taken:  0.4293074607849121
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=600,
       n_clusters=10, n_init=5, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)
-136.9407598765433


In [31]:
K_Means(wine_data, wine_target, random_grid)

C:\Users\miles\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


time taken:  0.3847661018371582
KMeans(algorithm='full', copy_x=True, init='k-means++', max_iter=600,
       n_clusters=10, n_init=2, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)
-1430808.932699262


C:\Users\miles\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [32]:
K_Means(bcancer_data, bcancer_target, random_grid)

C:\Users\miles\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


time taken:  0.630089521408081
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
       n_clusters=10, n_init=2, n_jobs=-1, precompute_distances=True,
       random_state=None, tol=0.0001, verbose=0)
-4243492.788923874


C:\Users\miles\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
